In [2]:
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models import VocabTransform

from common import *
import os
import glob
from joblib import Parallel, delayed
import itertools
import multiprocessing
import copy 

from glob import glob

cpu_count = multiprocessing.cpu_count() -1
DATA_FOLDER = '../data/'

In [4]:
flist = glob(join(DATA_FOLDER, 'documents/*.json.gz'))

In [5]:
class DocumentCorpus(gensim.corpora.TextCorpus):
    def get_texts(self):
        for fn in self.input: # for each relevant file
            with GzipFile(fn, 'r') as myzip:
                text = myzip.read()
            docs = json.loads(text)
            for doc in docs:
                yield sum(doc, [])

                
def save_corpus(ziped_files, dir_name, prefix):
    corpus = DocumentCorpus(ziped_files)
    
    dic_name = join(dir_name, '%s.dict' % prefix)
    corp_name = join(dir_name, '%s_corpus.mm' % prefix)
    
    corpus.dictionary.save(dic_name)
    corpora.MmCorpus.serialize(corp_name, corpus)
    
    return dic_name, corp_name
                

In [ ]:
save_corpus(flist, DATA_FOLDER, 'pure')

### Filter out tokens that appear in less than 5 documents (absolute number) or more than 50% documents

In [ ]:
%%time
# filter the dictionary
old_dict = corpora.Dictionary.load(join(DATA_FOLDER, 'old.dict'))
new_dict = copy.deepcopy(old_dict)
new_dict.filter_extremes(no_below=3, keep_n=None)
new_dict.save(join(DATA_FOLDER, 'filtered.dict'))

In [ ]:
%%time
# now transform the corpus
corpus = corpora.MmCorpus(join(DATA_FOLDER, 'corpus.mm'))
old2new = {old_dict.token2id[token]:new_id for new_id, token in new_dict.iteritems()}
vt = VocabTransform(old2new)

In [ ]:
%%time
corpora.MmCorpus.serialize(join(DATA_FOLDER, 'filtered_corpus.mm'), vt[corpus], id2word=new_dict)

In [55]:
!cp ../data/*corpus* ~/Yandex.Disk
!cp ../data/*.dict ~/Yandex.Disk
!cp ../data/all_docs.txt ~/Yandex.Disk

cp: cannot stat '../data/*corpus*': No such file or directory
cp: cannot stat '../data/*.dict': No such file or directory
cp: cannot stat '../data/all_docs.txt': No such file or directory
